In [1]:
import re
import pymorphy3

from joblib import dump, load

In [2]:
special_characters = ['</p>', '\xa0', '<br/>', '\n', '-&gt;']

In [3]:
stop_words = ['в', 'на', 'с', 'и', 'я', 'о', 'к', 'а', 'за', 'по',
              'об', 'у', 'бы', 'от', 'ли', 'ул', 'что', 'со', 'из',
              'для', 'про', 'г', 'до', 'то', 'быть', 'по', 'мочь',
              'январь','февраль','март','апрель','май','июнь','июль',
              'август','сентябрь','октябрь','ноябрь','декабрь','год',
              'стать','это','когда','даже', 'янв', 'фев', 'апр', 'авг',
              'сен', 'окт', 'ноя', 'дек', 'ые', 'ый', 'ым', 'ых', 'ая']


In [4]:
def cleansing(text):
    res = []
    for i in range(len(text)):
        res.append(text[i])
        for sp_ch in special_characters:
            res[i] = (text[i].replace(sp_ch,' '))
        res[i] = res[i].lower()
        res[i] = re.sub(r'[^\w\s]', ' ', res[i])
        res[i] = re.sub(r'[^\w\s]+|[\d]+', r' ',res[i]).strip()
        res[i] = re.sub('[a-zA-Z\s]+', ' ', res[i])
        res[i] = re.sub(" +", " ", res[i])
    return res

morph = pymorphy3.MorphAnalyzer()

def lemmatize(sentence):
    words = sentence.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
    return res

def delete_stop_words(sentence):
    res = list()
    for word in sentence:
        if word not in stop_words:
            res.append(word)
    return res

def join(sentence):
    return ' '.join(sentence)

def processing_pipeline(reviews):
    cleansed_reviews = cleansing(reviews)
    lemmatized_reviews = list(map(lemmatize,cleansed_reviews))
    clean_reviews = list(map(delete_stop_words,lemmatized_reviews))
    final_reviews = list(map(join,clean_reviews))
    return final_reviews


In [29]:
review = ['В марте оформляла кредитную карту год без % по ссылке друга. Не думала, что получу одобрение, т.к. уже были открытые карточки в других банках, но предварительное одобрение было уже через минуту. Выбрала лимит, приняла звонок от сотрудника банка и договорилась, что приду сама за получением карты в офис у м. Сокольники :) Из плюсов хочу отметить минимум очередей, современность отделения и электронную очередь. Сама взяла быстро талон, и через минут 7 была у менеджера. Мне понадобился только паспорт для получения карты) Сотрудница предложила страховку по карте и я согласилась, т.к. комиссия списывается вроде как только если не гасить за льготный период долг, что очень выгодно. Обслуживание на все 100% понравилось, так как девушка в отделении была заинтересована в раскрытии всех условий карты, даже когда я особо и не спрашивала!']

In [30]:
X = processing_pipeline(review)
X

['оформлять кредитный карта без ссылка друг не думать получить одобрение так уже открытый карточка другой банк но предварительный одобрение уже через минута выбрать лимит принять звонок сотрудник банк договориться прийти сам получение карта офис м сокольник плюс хотеть отметить минимум очередь современность отделение электронный очередь сам взять быстро талон через минута менеджер понадобиться только паспорт получение карта сотрудница предложить страховка карта согласиться так комиссия списываться вроде как только если не гасить льготный период долг очень выгодно обслуживание всё понравиться так как девушка отделение заинтересовать раскрытие весь условие карта особо не спрашивать']

In [31]:
vectorizer = load('vectorizer.joblib')
x = vectorizer.transform([X[0]])

In [32]:
model = load('SVC_model.joblib')
y = model.predict(x)

In [33]:
y

array([1.])